In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 2_anisotropic_df_dispersions.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Use DF samples + N-body data to calculate and compare the dispersion and 
anisotropy profiles.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu
from astropy import constants as apc

## Analysis
import scipy.stats
import scipy.interpolate

## galpy
from galpy import orbit
from galpy import potential

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import kinematics as pkin
from tng_dfs import plot as pplot
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/5_compare_distribution_functions/2_anisotropic_df_dispersions/')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Functions

In [ ]:
def compute_mass_error_weighted_deviation_beta_vdisp(nbody_orbs, sample_orbs, 
    nbody_mass, n_bs=100, adaptive_binning_kwargs={}, 
    velocity_quantities_squared=False):
    '''compute_mass_error_weighted_deviation_beta_vdisp:
    
    Compute the mass- and uncertainty-weighted deviation of the N-body 
    velocity dispersion / beta trend from the DF samples. 

    For the binning scheme the default kwargs are:
    - n: min(500, number of N-body particles//10)
    - rmin: 0.
    - rmax: max(N-body particle radii)
    - bin_mode: 'exact numbers'
    - bin_equal_n: True
    - end_mode: 'ignore'
    - bin_cents_mode: 'median'

    Args:
        nbody_orbs (galpy.orbit.Orbit): N-body orbits
        sample_orbs (galpy.orbit.Orbit): DF samples
        nbody_mass (np.ndarray): N-body particle masses
        n_bs (int): Number of times to bootstrap the DF/N-body samples
            to compute the deviation statistic for error estimation
        adaptive_binning_kwargs (dict): kwargs for get_radius_binning(), will
            be populated with defaults listed above if not provided.
        velocity_quantities_squared (bool): If True, use the squared velocity 
            dispersions/mean squares that are output from 
            pkin.compute_betas_bootstrap(). If False, take the square root of
            these quantities.
    
    Returns:
        mwed_[beta,vr2,vp2,vt2] (np.ndarray): Mass-weighted error deviation
    '''
    # Binning for velocity dispersions and betas
    n_bin = np.min([500, len(nbody_orbs)//10]) # n per bin
    if 'n' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['n'] = n_bin
    if 'rmin' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['rmin'] = 0.
    if 'rmax' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['rmax'] = np.max( nbody_orbs.r().to_value(apu.kpc) )
    if 'bin_mode' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['bin_mode'] = 'exact numbers'
    if 'bin_equal_n' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['bin_equal_n'] = True
    if 'end_mode' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['end_mode'] = 'ignore'
    if 'bin_cents_mode' not in adaptive_binning_kwargs.keys():
        adaptive_binning_kwargs['bin_cents_mode'] = 'median'

    bin_edges, bin_cents, _ = pkin.get_radius_binning(nbody_orbs, 
        **adaptive_binning_kwargs)
    bin_size = bin_edges[1:] - bin_edges[:-1]

    # Compute velocity dispersions for N-body
    compute_betas_kwargs = {'use_dispersions':True,
                            'return_kinematics':True}
    nbody_beta, nbody_vr2, nbody_vp2, nbody_vt2 = \
        pkin.compute_betas_bootstrap(nbody_orbs, bin_edges, n_bootstrap=n_bs, 
        compute_betas_kwargs=compute_betas_kwargs)

    # Compute velocity dispersions for the DF samples
    compute_betas_kwargs = {'use_dispersions':True,
                            'return_kinematics':True}
    sample_beta, sample_vr2, sample_vp2, sample_vt2 = \
        pkin.compute_betas_bootstrap(sample_orbs, bin_edges, n_bootstrap=n_bs, 
        compute_betas_kwargs=compute_betas_kwargs)

    if not velocity_quantities_squared:
        nbody_vr2 = np.sqrt(nbody_vr2)
        nbody_vp2 = np.sqrt(nbody_vp2)
        nbody_vt2 = np.sqrt(nbody_vt2)
        sample_vr2 = np.sqrt(sample_vr2)
        sample_vp2 = np.sqrt(sample_vp2)
        sample_vt2 = np.sqrt(sample_vt2)

    # Compute the mass profile for the N-body data
    mass_profile = np.zeros(len(bin_cents))
    rs = nbody_orbs.r().to_value(apu.kpc)
    for i in range(len(bin_cents)):
        mass_profile[i] = np.sum(nbody_mass[(rs > bin_edges[i]) &\
                                            (rs < bin_edges[i+1])])

    # Compute the inter-sigma range for the N-body data, which will be the error
    nbody_beta_err = np.percentile(nbody_beta, 84, axis=0) - \
                     np.percentile(nbody_beta, 16, axis=0)
    nbody_vr2_err = np.percentile(nbody_vr2, 84, axis=0) - \
                    np.percentile(nbody_vr2, 16, axis=0)
    nbody_vp2_err = np.percentile(nbody_vp2, 84, axis=0) - \
                    np.percentile(nbody_vp2, 16, axis=0)
    nbody_vt2_err = np.percentile(nbody_vt2, 84, axis=0) - \
                    np.percentile(nbody_vt2, 16, axis=0)

    # Compute the mass-error-weighted deviation between the N-body and DF 
    # sample trends
    mewd_beta = np.sum( np.abs(nbody_beta - sample_beta)*\
                        mass_profile/nbody_beta_err, axis=1 )/\
                np.sum(mass_profile)
    mewd_vr2 = np.sum( np.abs(nbody_vr2 - sample_vr2)*\
                        mass_profile/nbody_vr2_err, axis=1 )/\
                np.sum(mass_profile)
    mewd_vp2 = np.sum( np.abs(nbody_vp2 - sample_vp2)*\
                        mass_profile/nbody_vp2_err, axis=1 )/\
                np.sum(mass_profile)
    mewd_vt2 = np.sum( np.abs(nbody_vt2 - sample_vt2)*\
                        mass_profile/nbody_vt2_err, axis=1 )/\
                np.sum(mass_profile)

    return mewd_beta, mewd_vr2, mewd_vp2, mewd_vt2

### Compute the dispersions

In [ ]:
verbose = True

# Pathing
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Anisotropy profile calculation keywords
n_bs = 100

# Get the sample orbits
sample_data_cb = np.load(os.path.join(analysis_dir,'sample_data_cb.npy'),
    allow_pickle=True)
sample_data_om = np.load(os.path.join(analysis_dir,'sample_data_om.npy'),
    allow_pickle=True)
sample_data_om2 = np.load(os.path.join(analysis_dir,'sample_data_om2.npy'),
    allow_pickle=True)

mewd_data_cb = []
mewd_data_om = []
mewd_data_om2 = []

for i in range(n_mw):
    # if i != 1: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'Calculating anisotropy profiles for MW analog '
                          f'{i+1}/{n_mw}, merger {j+1}/{n_major}', end='\r')

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]
        orbs = co.get_orbs('stars')[indx]
        n_star = len(orbs)
        star_mass = co.get_masses('stars')[indx].to_value(apu.Msun)
        r = orbs.r().to_value(apu.kpc)
        r_softening = putil.get_softening_length('stars', z=0, physical=True)
        rmin = np.max([np.min(r), r_softening])

        ### Compute the mass-weighted error deviation for constant beta ###
        mask = (sample_data_cb['z0_sid'] == z0_sid) &\
               (sample_data_cb['major_acc_sid'] == major_acc_sid) &\
               (sample_data_cb['major_mlpid'] == major_mlpid) &\
               (sample_data_cb['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]

        sample = sample_data_cb['sample'][indx]

        adaptive_binning_kwargs = {'rmin':rmin}
        mewd = compute_mass_error_weighted_deviation_beta_vdisp(orbs, 
            sample, star_mass, n_bs=n_bs, 
            adaptive_binning_kwargs=adaptive_binning_kwargs, 
            velocity_quantities_squared=False)
        mewd_self = compute_mass_error_weighted_deviation_beta_vdisp(orbs,
            orbs, star_mass, n_bs=n_bs,
            adaptive_binning_kwargs=adaptive_binning_kwargs, 
            velocity_quantities_squared=False)

        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            mewd[0],
            mewd[1],
            mewd[2],
            mewd[3],
            mewd_self[0],
            mewd_self[1],
            mewd_self[2],
            mewd_self[3],
        )
        mewd_data_cb.append(_data)

        ### Compute the mass-weighted error deviation for Osipkov-Merritt ###
        mask = (sample_data_om['z0_sid'] == z0_sid) &\
               (sample_data_om['major_acc_sid'] == major_acc_sid) &\
               (sample_data_om['major_mlpid'] == major_mlpid) &\
               (sample_data_om['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]

        sample = sample_data_om['sample'][indx]

        adaptive_binning_kwargs = {'rmin':rmin}
        mewd = compute_mass_error_weighted_deviation_beta_vdisp(orbs, 
            sample, star_mass, n_bs=n_bs, 
            adaptive_binning_kwargs=adaptive_binning_kwargs, 
            velocity_quantities_squared=False)
        mewd_self = compute_mass_error_weighted_deviation_beta_vdisp(orbs,
            orbs, star_mass, n_bs=n_bs, 
            adaptive_binning_kwargs=adaptive_binning_kwargs, 
            velocity_quantities_squared=False)

        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            mewd[0],
            mewd[1],
            mewd[2],
            mewd[3],
            mewd_self[0],
            mewd_self[1],
            mewd_self[2],
            mewd_self[3],
        )
        mewd_data_om.append(_data)

        ### Compute the mass-weighted error deviation for OM2 ###
        mask = (sample_data_om2['z0_sid'] == z0_sid) &\
               (sample_data_om2['major_acc_sid'] == major_acc_sid) &\
               (sample_data_om2['major_mlpid'] == major_mlpid) &\
               (sample_data_om2['merger_number'] == j+1)
        indx = np.where(mask)[0]
        assert len(indx) == 1, 'Something went wrong'
        indx = indx[0]

        sample = sample_data_om2['sample'][indx]

        adaptive_binning_kwargs = {'rmin':rmin}
        mewd = compute_mass_error_weighted_deviation_beta_vdisp(orbs,
            sample, star_mass, n_bs=n_bs,
            adaptive_binning_kwargs=adaptive_binning_kwargs,
            velocity_quantities_squared=False)
        mewd_self = compute_mass_error_weighted_deviation_beta_vdisp(orbs,
            orbs, star_mass, n_bs=n_bs,
            adaptive_binning_kwargs=adaptive_binning_kwargs,
            velocity_quantities_squared=False)
        
        _data = (
            z0_sid,
            major_acc_sid,
            major_mlpid,
            j+1,
            mewd[0],
            mewd[1],
            mewd[2],
            mewd[3],
            mewd_self[0],
            mewd_self[1],
            mewd_self[2],
            mewd_self[3],
        )
        mewd_data_om2.append(_data)

# Save the data as a pickle
header = ['z0_sid','major_acc_sid','major_mlpid','merger_number',
          'mewd_beta','mewd_vr2','mewd_vp2','mewd_vt2',
          'mewd_self_beta','mewd_self_vr2','mewd_self_vp2','mewd_self_vt2']
with open(os.path.join(analysis_dir,'mewd_data_cb.pkl'),'wb') as handle:
    pickle.dump([header,mewd_data_cb], handle)
with open(os.path.join(analysis_dir,'mewd_data_om.pkl'),'wb') as handle:
    pickle.dump([header,mewd_data_om], handle)
with open(os.path.join(analysis_dir,'mewd_data_om2.pkl'),'wb') as handle:
    pickle.dump([header,mewd_data_om2], handle)

# Also save as a structured array
mewd_data_dtype = np.dtype([
    ('z0_sid',int),
    ('major_acc_sid',int),
    ('major_mlpid',int),
    ('merger_number',int),
    ('mewd_beta',object),
    ('mewd_vr2',object),
    ('mewd_vp2',object),
    ('mewd_vt2',object),
    ('mewd_self_beta',object),
    ('mewd_self_vr2',object),
    ('mewd_self_vp2',object),
    ('mewd_self_vt2',object)])
mewd_data_cb = np.array(mewd_data_cb, dtype=mewd_data_dtype)
mewd_data_om = np.array(mewd_data_om, dtype=mewd_data_dtype)
mewd_data_om2 = np.array(mewd_data_om2, dtype=mewd_data_dtype)
np.save(os.path.join(analysis_dir,'mewd_data_cb.npy'), mewd_data_cb)
np.save(os.path.join(analysis_dir,'mewd_data_om.npy'), mewd_data_om)
np.save(os.path.join(analysis_dir,'mewd_data_om2.npy'), mewd_data_om2)

### Get the stashed data

In [ ]:
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Or load the structured arrays
mewd_data_cb = np.load(os.path.join(analysis_dir,'mewd_data_cb.npy'),
    allow_pickle=True)
mewd_data_om = np.load(os.path.join(analysis_dir,'mewd_data_om.npy'),
    allow_pickle=True)
mewd_data_om2 = np.load(os.path.join(analysis_dir,'mewd_data_om2.npy'),
    allow_pickle=True)

# Load the merger information
merger_data = np.load(os.path.join(analysis_dir,'merger_data.npy'), 
    allow_pickle=True)

checks = True
if checks:
    assert np.all( mewd_data_cb['z0_sid'] == mewd_data_om['z0_sid'] ), \
        'Something went wrong'
    assert np.all( mewd_data_cb['major_acc_sid'] == mewd_data_om['major_acc_sid'] ), \
        'Something went wrong'
    assert np.all( mewd_data_cb['major_mlpid'] == mewd_data_om['major_mlpid'] ), \
        'Something went wrong'

In [ ]:
# Compute the typical MEWD value for the sample self-comparison, as well as the 
# 16th and 84th percentiles
mewd_keys = ['mewd_self_beta','mewd_self_vr2','mewd_self_vp2','mewd_self_vt2']
mwd_cb_self_percs = np.zeros((4,3))
for i in range(4):
    p = np.array([])
    for j in range(n_mw):
        p = np.append(p, mewd_data_cb[mewd_keys[i]][j])
    mwd_cb_self_percs[i] = np.percentile(p, [16,50,84])

mwd_om_self_percs = np.zeros((4,3))
for i in range(4):
    p = np.array([])
    for j in range(n_mw):
        p = np.append(p, mewd_data_om[mewd_keys[i]][j])
    mwd_om_self_percs[i] = np.percentile(p, [16,50,84])

mwd_om2_self_percs = np.zeros((4,3))
for i in range(4):
    p = np.array([])
    for j in range(n_mw):
        p = np.append(p, mewd_data_om2[mewd_keys[i]][j])
    mwd_om2_self_percs[i] = np.percentile(p, [16,50,84])

### Make the paper figure

In [ ]:
# Plot keywords
columnwidth, textwidth = pplot.get_latex_columnwidth_textwidth_inches()
key_suffixes = ['beta','vr2','vp2','vt2']
ticklabel_fs = 6
label_fs = 10
# facecolor='none'
marker_s = 10
marker_linewidth = 0.5
marker_zorder = 4
marker_edgecolor='Black'
data_range = [0.4,20]
panel_labels = [r'$\beta$', r'$\sigma_{r}$', r'$\sigma_{\phi}$', 
    r'$\sigma_{\theta}$']
panel_label_fs = 10
one_to_one_line_numbers = True

# Colormap for constant anisotropy
cmap = pplot.colors().colourmap('rainbow')
norm = mpl.colors.Normalize(vmin=-0.1, vmax=1.0)

# Make the figure
fig = plt.figure(figsize=(textwidth,3.5))
axs = fig.subplots(nrows=2, ncols=4)
n_merger = len(merger_data)

# Plot the panels, top row CB vs OM, bottom row OM2 vs OM
for i in range(n_merger):
    # Color is the value of the anisotropy
    anisotropy = merger_data['anisotropy'][i]
    c = cmap(norm(anisotropy))
    if anisotropy > 0.5:
        this_zorder = marker_zorder + 1
    else:
        this_zorder = marker_zorder - 1
    # Loop over beta, vr2, vp2, vt2
    for j in range(4):
        key = f'mewd_{key_suffixes[j]}'
        axs[0,j].scatter(np.nanmedian(mewd_data_cb[key][i]),
                         np.nanmedian(mewd_data_om[key][i]),
                         marker='o', s=marker_s, zorder=this_zorder, 
                         linewidth=marker_linewidth, facecolor=c, 
                         edgecolor=marker_edgecolor)
        axs[1,j].scatter(np.nanmedian(mewd_data_cb[key][i]),
                         np.nanmedian(mewd_data_om2[key][i]),
                         marker='o', s=marker_s, zorder=this_zorder, 
                         linewidth=marker_linewidth, facecolor=c, 
                         edgecolor=marker_edgecolor)

# Assess the typical errors
mewd_cb_self_percs = np.zeros((4,3))
mewd_om_self_percs = np.zeros((4,3))
mewd_om2_self_percs = np.zeros((4,3))
key_suffixes = ['beta','vr2','vp2','vt2']

for i in range(4):
    key = 'mewd_self_'+key_suffixes[i]
    p_cb = np.array([])
    p_om = np.array([])
    p_om2 = np.array([])
    for j in range(n_merger):
        p_cb = np.concatenate( (p_cb, mewd_data_om[key][j]) )
        p_om = np.concatenate( (p_om, mewd_data_om2[key][j]) )
        p_om2 = np.concatenate( (p_om2, mewd_data_om2[key][j]) )
    mewd_cb_self_percs[i] = np.nanpercentile(p_cb, [16,50,84])
    mewd_om_self_percs[i] = np.nanpercentile(p_om, [16,50,84])
    mewd_om2_self_percs[i] = np.nanpercentile(p_om2, [16,50,84])

# Labels and limits
axs[0,0].set_ylabel(r'$\delta_\mathrm{OM}$', fontsize=label_fs)
axs[1,0].set_ylabel(r'$\delta_\mathrm{OM2}$', fontsize=label_fs)
for i in range(4):
    # Bottom row has x-axis labels
    axs[1,i].set_xlabel(r'$\delta_\mathrm{CB}$', fontsize=label_fs)

    # Limits
    axs[0,i].set_xlim(data_range)
    axs[0,i].set_ylim(data_range)
    axs[1,i].set_xlim(data_range)
    axs[1,i].set_ylim(data_range)

    # Panel labels
    axs[0,i].text(0.2, 0.9, panel_labels[i], transform=axs[0,i].transAxes,
        ha='left', va='top', fontsize=panel_label_fs, 
        bbox=dict(facecolor='white', edgecolor='Black'))
    axs[1,i].text(0.2, 0.9, panel_labels[i], transform=axs[1,i].transAxes,
        ha='left', va='top', fontsize=panel_label_fs,
        bbox=dict(facecolor='white', edgecolor='Black'))

    # Scale and ticks
    for j in range(2):
        axs[j,i].set_xscale('log')
        axs[j,i].set_yscale('log')
        axs[j,i].tick_params(axis='both', labelsize=ticklabel_fs)
        if i > 0:
            axs[j,i].tick_params(labelleft=False)
    axs[0,i].tick_params(labelbottom=False)

    # Self-consistent errors
    axs[0,i].axline(xy1=[0,0], xy2=[1,1], color='k', ls='--')
    axs[0,i].axvline(mewd_cb_self_percs[i,1], color='Black', ls='solid')
    axs[0,i].axvspan(mewd_cb_self_percs[i,0], mewd_cb_self_percs[i,2], 
        color='Black', alpha=0.25)
    axs[0,i].axhline(mewd_om_self_percs[i,1], color='Black', ls='solid')
    axs[0,i].axhspan(mewd_om_self_percs[i,0], mewd_om_self_percs[i,2], 
        color='Black', alpha=0.25)
    for fac in [2,4]:
        axs[0,i].axvline(fac*mewd_cb_self_percs[i,1], color='Gray', 
            ls='dotted')
        axs[0,i].axhline(fac*mewd_om_self_percs[i,1], color='Gray', 
            ls='dotted')
    
    axs[1,i].axline(xy1=[0,0], xy2=[1,1], color='k', ls='--')
    axs[1,i].axvline(mewd_cb_self_percs[i,1], color='Black', ls='solid')
    axs[1,i].axvspan(mewd_cb_self_percs[i,0], mewd_cb_self_percs[i,2], 
        color='Black', alpha=0.25)
    axs[1,i].axhline(mewd_om2_self_percs[i,1], color='Black', ls='solid')
    axs[1,i].axhspan(mewd_om2_self_percs[i,0], mewd_om2_self_percs[i,2], 
        color='Black', alpha=0.25)
    for fac in [2,4]:
        axs[1,i].axvline(fac*mewd_cb_self_percs[i,1], color='Gray', 
            ls='dotted')
        axs[1,i].axhline(fac*mewd_om2_self_percs[i,1], color='Gray', 
            ls='dotted')
    
    # One-to-one line
    if one_to_one_line_numbers:
        n_cb = 0
        n_om = 0
        key = 'mewd_'+key_suffixes[i]
        for j in range(n_merger):
            om_gt_cb = np.nanmedian( mewd_data_om[key][j] ) > \
                       np.nanmedian( mewd_data_cb[key][j] )
            if om_gt_cb: # Opposite because greater means worse
                n_cb += 1
            else:
                n_om += 1
        axs[0,i].text(0.925, 0.825, str(n_om), transform=axs[0,i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)
        axs[0,i].text(0.825, 0.925, str(n_cb), transform=axs[0,i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)

        n_cb = 0
        n_om2 = 0
        key = 'mewd_'+key_suffixes[i]
        for j in range(n_merger):
            om2_gt_cb = np.median( mewd_data_om2[key][j] ) > \
                        np.median( mewd_data_cb[key][j] )
            if om2_gt_cb: # Opposite because greater means worse
                n_cb += 1
            else:
                n_om2 += 1
        axs[1,i].text(0.925, 0.825, str(n_om2), transform=axs[1,i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)
        axs[1,i].text(0.825, 0.925, str(n_cb), transform=axs[1,i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)

# Colorbar
cax = fig.add_axes([0.92, 0.2, 0.02, 0.7])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_label(r'$\beta$', fontsize=label_fs)
cbar.ax.tick_params(labelsize=ticklabel_fs)

fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1, right=0.91)
fig.savefig(os.path.join(local_fig_dir,'delta_comparison.pdf'), dpi=300, 
    bbox_inches='tight')
fig.savefig(os.path.join(local_fig_dir,'delta_comparison.png'), dpi=300, 
    bbox_inches='tight')
fig.show()

### Also make the figures individually

These are older plots than the one above, which has been updated to use space more efficiently.

In [ ]:
# Column-sized figure width
columnwidth = 244./72.27 # In inches, from pt
# Full-sized figure width
textwidth = 508./72.27 # In inches, from pt

# Plot params
ylabels = [r'$\beta$', r'$\sigma_{r},$', r'$\sigma_{\phi},$', r'$\sigma_{\theta},$']
label_fs = 9
key_suffixes = ['beta','vr2','vp2','vt2']
# beta_ticks = [0,2,4,6,8,10]
beta_range = [0.4,20]
# vdisp_ticks = [0,2,4,6,8,10,12,14,16]
vdisp_range = [0.4,20]
# ticks = [beta_ticks, vdisp_ticks, vdisp_ticks, vdisp_ticks]
ticklabel_fs = 6
# facecolor='none'
edgecolor='Black'
s = 12
one_to_one_line_numbers = True

fig = plt.figure(figsize=(textwidth, 2.0))
axs = fig.subplots(nrows=1, ncols=4)

cmap = pplot.colors().colourmap('rainbow')
norm = mpl.colors.Normalize(vmin=-0.1, vmax=1.0)

# Plot the data
n_merger = len(merger_data)
for i in range(n_merger):
    # if merger_data['anisotropy'][i] > 0.8:
    #     edgecolor = 'Red'
    #     zorder = 4
    # # else:
    # #     edgecolor = 'Black'
    # #     zorder = 1
    # elif np.abs(merger_data['anisotropy'][i]) < 0.2:
    #     edgecolor = 'MediumBlue'
    #     zorder = 3
    # else:
    #     edgecolor = 'Black'
    #     zorder = 1

    c = cmap(norm(merger_data['anisotropy'][i]))

    for j in range(4):
        
        key = 'mewd_'+key_suffixes[j]
        axs[j].scatter(np.median( mewd_data_om[key][i] ), 
                       np.median( mewd_data_cb[key][i] ),
                       facecolor=c, edgecolor=edgecolor, s=s,
                       marker='o', zorder=3, linewidth=0.5)

# Colorbar
cax = fig.add_axes([0.92, 0.23, 0.02, 0.7])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_label(r'$\beta$', fontsize=label_fs)
cbar.ax.tick_params(labelsize=ticklabel_fs)

# Assess the typical errors for the self-consistent case
mewd_cb_self_percs = np.zeros((4,3))
mewd_om_self_percs = np.zeros((4,3))
key_suffixes = ['beta','vr2','vp2','vt2']

n_merger = len(merger_data)
for i in range(4):
    key = 'mewd_self_'+key_suffixes[i]
    p_cb = np.array([])
    p_om = np.array([])
    for j in range(n_merger):
        p_cb = np.concatenate( (p_cb, mewd_data_om[key][j]) )
        p_om = np.concatenate( (p_om, mewd_data_om2[key][j]) )
    mewd_cb_self_percs[i] = np.percentile(p_cb, [16,50,84])
    mewd_om_self_percs[i] = np.percentile(p_om, [16,50,84])

# Labels and limits
for i in range(4):
    axs[i].set_xlabel(r'$\delta($'+ylabels[i]+r'$_{\mathrm{OM}})$',
        fontsize=label_fs)
    axs[i].set_ylabel(r'$\delta($'+ylabels[i]+r'$_{\mathrm{CB}})$',
        fontsize=label_fs)
    if i == 0:
        axs[i].set_xlim(beta_range)
        axs[i].set_ylim(beta_range)
    else:
        axs[i].set_xlim(vdisp_range)
        axs[i].set_ylim(vdisp_range)

    # Self-consistent errors
    axs[i].axline(xy1 = [0,0], slope=1., color='k', ls='--')
    axs[i].axvline(mewd_om_self_percs[i,1], color='Black', ls='solid')
    axs[i].axvspan(mewd_om_self_percs[i,0], mewd_om_self_percs[i,2], 
        color='Black', alpha=0.25)
    axs[i].axhline(mewd_cb_self_percs[i,1], color='Black', ls='solid')
    axs[i].axhspan(mewd_cb_self_percs[i,0], mewd_cb_self_percs[i,2], 
        color='Black', alpha=0.25)
    for fac in [2,4]:
        axs[i].axvline(fac*mewd_om_self_percs[i,1], color='Gray', 
            ls='dotted')
        axs[i].axhline(fac*mewd_cb_self_percs[i,1], color='Gray', 
            ls='dotted')
        
    # Scale and ticks
    axs[i].set_xscale('log')
    axs[i].set_yscale('log')
    axs[i].xaxis.set_major_formatter(plt.FormatStrFormatter('%.0f'))
    axs[i].yaxis.set_major_formatter(plt.FormatStrFormatter('%.0f'))
    axs[i].tick_params(axis='both', labelsize=ticklabel_fs)
    # axs[i].set_xticks(ticks[i])
    # axs[i].set_yticks(ticks[i])

    # One-to-one line
    if one_to_one_line_numbers:
        n_cb = 0
        n_om = 0
        key = 'mewd_'+key_suffixes[i]
        for j in range(n_merger):
            cb_gt_om = np.median( mewd_data_om[key][j] ) > \
                       np.median( mewd_data_cb[key][j] )
            if cb_gt_om:
                n_cb += 1
            else:
                n_om += 1

        axs[i].text(0.925, 0.825, str(n_om), transform=axs[i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)
        axs[i].text(0.825, 0.925, str(n_cb), transform=axs[i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)


fig.tight_layout()
fig.subplots_adjust(wspace=0.3, right=0.91)
fig.savefig('./fig/delta_comparison_om_cb.pdf', dpi=300)
fig.show()

In [ ]:
# Column-sized figure width
columnwidth = 244./72.27 # In inches, from pt
# Full-sized figure width
textwidth = 508./72.27 # In inches, from pt

# Plot params
ylabels = [r'$\beta$', r'$\sigma_{r},$', r'$\sigma_{\phi},$', r'$\sigma_{\theta},$']
label_fs = 9
key_suffixes = ['beta','vr2','vp2','vt2']
# beta_ticks = [0,2,4,6,8,10]
beta_range = [0.4,20]
# vdisp_ticks = [0,2,4,6,8,10,12,14,16]
vdisp_range = [0.4,20]
# ticks = [beta_ticks, vdisp_ticks, vdisp_ticks, vdisp_ticks]
ticklabel_fs = 6
# facecolor='none'
edgecolor='Black'
s = 12
one_to_one_line_numbers = True

fig = plt.figure(figsize=(textwidth, 2.0))
axs = fig.subplots(nrows=1, ncols=4)

cmap = pplot.colors().colourmap('rainbow')
norm = mpl.colors.Normalize(vmin=-0.1, vmax=1.0)

# Plot the data
n_merger = len(merger_data)
for i in range(n_merger):
    # if merger_data['anisotropy'][i] > 0.8:
    #     edgecolor = 'Red'
    #     zorder = 4
    # # else:
    # #     edgecolor = 'Black'
    # #     zorder = 1
    # elif np.abs(merger_data['anisotropy'][i]) < 0.2:
    #     edgecolor = 'MediumBlue'
    #     zorder = 3
    # else:
    #     edgecolor = 'Black'
    #     zorder = 1

    c = cmap(norm(merger_data['anisotropy'][i]))

    for j in range(4):
        
        key = 'mewd_'+key_suffixes[j]
        axs[j].scatter(np.median( mewd_data_om[key][i] ), 
                       np.median( mewd_data_om2[key][i] ),
                       facecolor=c, edgecolor=edgecolor, s=s,
                       marker='o', zorder=3, linewidth=0.5)

# Colorbar
cax = fig.add_axes([0.92, 0.23, 0.02, 0.7])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_label(r'$\beta$', fontsize=label_fs)
cbar.ax.tick_params(labelsize=ticklabel_fs)

# Assess the typical errors for the self-consistent case
mewd_om2_self_percs = np.zeros((4,3))
mewd_om_self_percs = np.zeros((4,3))
key_suffixes = ['beta','vr2','vp2','vt2']

n_merger = len(merger_data)
for i in range(4):
    key = 'mewd_self_'+key_suffixes[i]
    p_om2 = np.array([])
    p_om = np.array([])
    for j in range(n_merger):
        p_om2 = np.concatenate( (p_om2, mewd_data_om[key][j]) )
        p_om = np.concatenate( (p_om, mewd_data_om2[key][j]) )
    mewd_om2_self_percs[i] = np.percentile(p_om2, [16,50,84])
    mewd_om_self_percs[i] = np.percentile(p_om, [16,50,84])

# Labels and limits
for i in range(4):
    axs[i].set_xlabel(r'$\delta($'+ylabels[i]+r'$_{\mathrm{OM}})$',
        fontsize=label_fs)
    axs[i].set_ylabel(r'$\delta($'+ylabels[i]+r'$_{\mathrm{OM2}})$',
        fontsize=label_fs)
    if i == 0:
        axs[i].set_xlim(beta_range)
        axs[i].set_ylim(beta_range)
    else:
        axs[i].set_xlim(vdisp_range)
        axs[i].set_ylim(vdisp_range)

    # Self-consistent errors
    axs[i].axline(xy1 = [0,0], slope=1., color='k', ls='--')
    axs[i].axvline(mewd_om_self_percs[i,1], color='Black', ls='solid')
    axs[i].axvspan(mewd_om_self_percs[i,0], mewd_om_self_percs[i,2], 
        color='Black', alpha=0.25)
    axs[i].axhline(mewd_om2_self_percs[i,1], color='Black', ls='solid')
    axs[i].axhspan(mewd_om2_self_percs[i,0], mewd_om2_self_percs[i,2], 
        color='Black', alpha=0.25)
    for fac in [2,4]:
        axs[i].axvline(fac*mewd_om_self_percs[i,1], color='Gray', 
            ls='dotted')
        axs[i].axhline(fac*mewd_om2_self_percs[i,1], color='Gray', 
            ls='dotted')
        
    # Scale and ticks
    axs[i].set_xscale('log')
    axs[i].set_yscale('log')
    axs[i].xaxis.set_major_formatter(plt.FormatStrFormatter('%.0f'))
    axs[i].yaxis.set_major_formatter(plt.FormatStrFormatter('%.0f'))
    axs[i].tick_params(axis='both', labelsize=ticklabel_fs)
    # axs[i].set_xticks(ticks[i])
    # axs[i].set_yticks(ticks[i])

    # One-to-one line
    if one_to_one_line_numbers:
        n_om2 = 0
        n_om = 0
        key = 'mewd_'+key_suffixes[i]
        for j in range(n_merger):
            om2_gt_om = np.median( mewd_data_om[key][j] ) > \
                        np.median( mewd_data_om2[key][j] )
            if om2_gt_om:
                n_om2 += 1
            else:
                n_om += 1

        axs[i].text(0.925, 0.825, str(n_om), transform=axs[i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)
        axs[i].text(0.825, 0.925, str(n_om2), transform=axs[i].transAxes, 
            ha='center', va='center', fontsize=ticklabel_fs)


fig.tight_layout()
fig.subplots_adjust(wspace=0.3, right=0.91)
fig.savefig('./fig/delta_comparison_om_om2.pdf', dpi=300)
fig.show()